In [1]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
DATA_DIRECTORY = "data"
STANDINGS_DIRECTORY = os.path.join(DATA_DIRECTORY, "standings")
SCORES_DIRECTORY = os.path.join(DATA_DIRECTORY, "scores")

In [24]:
years = list(range(2018, 2023))
months = ["october",
          "november",
          "december",
          "january",
          "february",
          "march",
          "april",
          "may",
          "june"]

url_start = "https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"

for year in years:
    for month in months:
        url = url_start.format(year=year, month=month)
        html_data = requests.get(url)

        with open("data/standings/{year}_{month}_games.html".format(year=year, month=month), "w+", encoding="utf-8") as f:
            f.write(html_data.text)

In [4]:
def get_boxscore_html_month(sleepTime=4, fileIndex=-1, files=os.listdir(STANDINGS_DIRECTORY)):
    # Membuka data HTML bulan & tahun tertentu dari folder 'standings'
    with open("data/standings/{filename}".format(filename=files[fileIndex]), "r", encoding="utf-8") as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [link.get("href") for link in links]
    box_scores = [f"https://www.basketball-reference.com{link}" for link in hrefs if link and "boxscore" in link and ".html" in link]

    # Men-downlaod tabel box score
    for index in range(len(box_scores)):
        boxscore_url = box_scores[index]
        save_path = os.path.join(SCORES_DIRECTORY, boxscore_url.split("/")[-1])

        html = requests.get(boxscore_url)

        soup = BeautifulSoup(html.text, 'html.parser')
        with open(save_path, "w+", encoding="utf-8") as f:
            f.write(str(html.text))
        time.sleep(sleepTime)
    
    print(f"File '{files[fileIndex]}' selesai.")

In [6]:
# def get_boxscore_html_month_test(sleepTime=4, fileIndex=-1, files=os.listdir(STANDINGS_DIRECTORY)):
#     # Membuka data HTML bulan & tahun tertentu dari folder 'standings'
#     with open("data/standings/{filename}".format(filename=files[fileIndex]), "r", encoding="utf-8") as f:
#         html = f.read()

#     soup = BeautifulSoup(html)
#     links = soup.find_all("a")
#     hrefs = [link.get("href") for link in links]
#     box_scores = [f"https://www.basketball-reference.com{link}" for link in hrefs if link and "boxscore" in link and ".html" in link]

#     # Men-downlaod tabel box score
#     for index in range(len(box_scores)):
#         boxscore_url = box_scores[index]
#         save_path = os.path.join(SCORES_DIRECTORY, boxscore_url.split("/")[-1])

#         html = requests.get(boxscore_url)

#         soup = BeautifulSoup(html.text, 'html.parser')
#         # with open(save_path, "w+", encoding="utf-8") as f:
#         #     f.write(str(html.text))
#         content_element = soup.find("div", id="content")

#         if content_element:
#             with open(os.path.join(os.path.join(DATA_DIRECTORY, "scores_cancel"), boxscore_url.split("/")[-1]), "w+", encoding="utf-8") as f:
#                 f.write(str(content_element))
#         else:
#             print(f"Elemen id=\"content\" tidak ditemukan pada{index}:{boxscore_url}")

#         # jeda agar request rate tidak terlalu besar
#         time.sleep(sleepTime)
    
#     print(f"File '{files[fileIndex]}' selesai.")

In [ ]:
startIndex = 1 # 36 file sudah selesai, lanjut ke 37 (37 tdk ada --> sudah selesai)
files = os.listdir(STANDINGS_DIRECTORY)[startIndex:]

for i in range(len(os.listdir(STANDINGS_DIRECTORY))-1):
    get_boxscore_html_month(sleepTime=3.5, fileIndex=i, files=files)